Like routing, we can also ask the LLM to select the most appropriate tool to use.

In this example, we see how to do Function/Tool Calling.

In essence, the LLM decides on the tool/function it wants to use and send back the function name and arguments for us to run on our own box, get the result and then add this to the messages in the next LLM request.


In [25]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import requests

In [26]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:10]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-y-


In [27]:
client = OpenAI()
MODEL = "gpt-4o-mini"

In [28]:
# original propmt
system_message = """
You are an assistant that is very good at determining what tool to use to solve queries.
"""

ai_programming = """

# TOOLS

You have two tools:

1. A calculator tool that can perform basic arithmetic operations, such as addition, subtraction, multiplication, and division. If this tool is used then respond in JSON FORMAT.

## Example JSON format:

{"tool": "calculator", "next": "do_calculation", "arguments": {"num1": 5, "num2": 5, "operation": "addition"}} 

2. A jokes tool that can provide a light-hearted joke for the audience reqested. If this tool is used then respond in JSON FORMAT.

## Example JSON format:

{"tool": "joke", "next": "do_joke", "audience": "Pythonistas" }} 

    
Thank you for your being accurate and complete with this query.
"""

system_message += ai_programming

In [29]:
# user_prompt = "What is 102 plus 3?"
user_prompt = "Tell me a joke when I am doing stand up at a Builders Conference"

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt},
]

response = client.chat.completions.create(model=MODEL, messages=prompts)

print(response.choices[0].message.content)

output = response.choices[0].message.content.replace("\n", "")

{"tool": "joke", "next": "do_joke", "audience": "Builders Conference"}


Having got the function and arguments, we extract these from the response and run it.

In [30]:
json_output = output.strip()
json_output = json.loads(json_output)
print(json_output)
do_next = json_output["next"]

{'tool': 'joke', 'next': 'do_joke', 'audience': 'Builders Conference'}


Now we use our day to day Python to run the selected tool.

There are other ways we might code this but this will suffice for demonstration purposes.

In [31]:
if do_next == "do_calculation":
    tool = json_output["tool"]
    num1 = json_output["arguments"]["num1"]
    num2 = json_output["arguments"]["num2"]
    operation = json_output["arguments"]["operation"]
    if operation == "addition":
        answer = num1 + num2
    elif operation == "subtraction":
        answer = num1 - num2
    elif operation == "multiplication":
        answer = num1 * num2
    elif operation == "division":
        answer = num1 / num2
    print(f"{num1} {operation} {num2} = {answer}")

In [32]:
if do_next == "do_joke":
    get_random_joke_internet = requests.get(
        "https://official-joke-api.appspot.com/random_joke"
    )

    print(get_random_joke_internet.json())

{'type': 'general', 'setup': 'What did the grape do when he got stepped on?', 'punchline': 'He let out a little wine.', 'id': 173}


Where are the functions run? From the OpenAI website, we can see at the arrow that tools are executed on our 'box'.

<img style="width:600px;" src="./images/where-tools-are-executed.png">
